### The below code performs

1. Renaming of the stimulus videos based on `Outcome`, `Temporality`, and `Anticipation`
2. Removes audio from the stimulus videos
3. Adds a black screen (lag) towards the end of the stimulus video (optional - in the final study, this step was not performed)
4. Calculate the video durations

In [1]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
from moviepy.editor import VideoFileClip
from moviepy.editor import *

Installing MoviePy - check out the documentation at - [LINK](https://zulko.github.io/moviepy/install.html)

The Library 'MoviePy' may not work having installed through pip. If that is the case, try installing it using 'Home Brew' for mac. 

In [2]:
files_to_ignore = ['.DS_Store']
stimulus_data_info_path = '../../stimulus_dataset/stimulus_dataset_information.xlsx'
df = pd.read_excel(stimulus_data_info_path)
df

,Stimulus Video Name,Outcome,Temporality,Anticipation,Brief Description,Human/Robot,Source,Shortlist,Final_Shortlist,Expected Outcomes,Comments,Final Video Name,Duration (s),When to Pause? (s)
0,ch5.mp4,good,sudden,unexpected,Human avoids being drenched in water from a wave,human,SuperBAD dataset,Y,N,NaN,NaN,good_unexp_sudden_1.mp4,8.70,NaN
1,cr4.mp4,good,gradual,expected,Robot turns on a light switch,robot,SuperBAD dataset,Y,Y,Bad?,I think people will see robot doing something ...,good_exp_gradual_2.mp4,14.12,NaN
2,fh1.mp4,bad,sudden,expected,Human falls trying to imitate pixar intro scene,human,SuperBAD dataset,M,N,NaN,not enough time to understand context,bad_sudden_exp_3.mp4,7.15,NaN
3,fh2.mp4,bad,sudden,expected,Human fails at performing box jumps,human,SuperBAD dataset,M,N,NaN,might be too obvious it goes bad,bad_sudden_exp_4.mp4,20.72,NaN
4,fh4.mp4,bad,sudden,unexpected,Falling off of a trampoline,human,SuperBAD dataset,M,N,NaN,NaN,bad_sudden_unexp_5.mp4,6.77,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,"Outcome Counts : {'bad': 30, 'good': 19}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,NaN,NaN,NaN,NaN,"Temporality Counts : {'gradual': 32, 'sudden':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
for index, row in df[49:60].iterrows():
#     print(row)
    outcome = row['Outcome']
    temporality = row['Temporality']
    anticipation = row['Anticipation']
    
    if anticipation == 'unexpected':
        anticipation = 'unexp'
    else:
        anticipation = 'exp'
    
    newVideoName = f"{outcome}_{temporality}_{anticipation}_{row['Stimulus Video Name']}"
    df.at[index, 'Final Video Name'] = newVideoName
    
    print(f"{row['Stimulus Video Name']}: {outcome}_{temporality}_{anticipation}")
df.to_csv('../stimulus_dataset/stimulus_dataset_information.csv', index=False)

50.mp4: good_gradual_exp
51.mp4: good_gradual_exp
52.mp4: good_gradual_exp
53.mp4: good_gradual_exp
54.mp4: good_gradual_exp
55.mp4: good_gradual_exp
56.mp4: good_gradual_exp
57.mp4: good_gradual_exp
ch2.mp4: good_gradual_exp
ch3.mp4: good_gradual_exp
ch4.mp4: good_gradual_exp


#### Rename videos

In [7]:
video_path = '../../stimulus_dataset/temp/'
videos = [video for video in os.listdir(video_path) if video not in files_to_ignore]
df = pd.read_csv('../../stimulus_dataset/stimulus_dataset_information.csv')

In [9]:
for video in sorted(videos):
    # Split the video file name by the dot to get the part before the extension
    filename_without_extension = video.split('.')[0]
    print(filename_without_extension)
    
    # Check if the part before the dot is an integer
    if filename_without_extension:
        for index, row in df[49:60].iterrows():
            newName = row['Final Video Name']
            if filename_without_extension == newName.split('.')[0].split('_')[-1]:
#                 print(filename_without_extension, newName)
                os.rename(f'../../stimulus_dataset/temp/{video}', f'../../stimulus_dataset/temp/{newName}')

50
51
52
53
54
55
56
57
ch2
ch3
ch4


#### Calculate stats

In [10]:
outcome_dict = df['Outcome'].value_counts().to_dict()
temporality_dict = df['Temporality'].value_counts().to_dict()
anticipation_dict = df['Anticipation'].value_counts().to_dict()
print(f'Outcome Counts : {outcome_dict}')
print(f'Temporality Counts : {temporality_dict}')
print(f'Anticipation Counts: {anticipation_dict}')

Outcome Counts : {'good': 30, 'bad': 30}
Temporality Counts : {'gradual': 43, 'sudden': 17}
Anticipation Counts: {'expected': 32, 'unexpected': 28}


## Audio Removal Processing

Change the path of the 'directory' appropriately to choose the videos based on the class of the videos to be edited

In [11]:
directory = '../../stimulus_dataset/temp/'
newDirectory = '../../stimulus_dataset/new_temp/'


if not os.path.exists(newDirectory):
    os.mkdir(newDirectory)

In [12]:
# The videos for which the audio has to be removed
videos = sorted([video for video in os.listdir(directory) if video not in files_to_ignore])
videos

['good_gradual_exp_50.mp4',
 'good_gradual_exp_51.mp4',
 'good_gradual_exp_52.mp4',
 'good_gradual_exp_53.mp4',
 'good_gradual_exp_54.mp4',
 'good_gradual_exp_55.mp4',
 'good_gradual_exp_56.mp4',
 'good_gradual_exp_57.mp4',
 'good_gradual_exp_ch2.mp4',
 'good_gradual_exp_ch3.mp4',
 'good_gradual_exp_ch4.mp4']

In [13]:
for video in videos:
#     print (element)
    videoClip = VideoFileClip(f"{directory}/{video}")
    noAudioClip = videoClip.without_audio()
    noAudioClip.write_videofile(f"{newDirectory}/{video}")

Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_50.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_50.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_50.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_51.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_51.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_51.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_52.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_52.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_52.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_53.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_53.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_53.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_54.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_54.mp4



...

Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_56.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_57.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_57.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_57.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_ch2.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_ch2.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_ch2.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_ch3.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_ch3.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_ch3.mp4
Moviepy - Building video ../stimulus_dataset/new_temp//good_gradual_exp_ch4.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//good_gradual_exp_ch4.mp4



Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//good_gradual_exp_ch4.mp4


## Lag addition - Black Screen

In [24]:
videos = sorted(os.listdir(newDirectory))
videos
# print(len(videos))
# print('\n'.join(sorted(videos)))

['good_gradual_exp_46.mp4',
 'good_gradual_exp_47.mp4',
 'good_gradual_exp_48.mp4',
 'good_gradual_exp_49.mp4']

If a .mp4 video file format is read and it is then converted or made any modifications to - to add the black screen, upon writing the file to local disk, the video files are corrupted - where the frames are frozen.

To fix this, try converting your input video to a different format or codec before processing it with moviepy. You can use a tool like FFmpeg to convert the video. For example, you can try converting the video to H.264 format using the following command: ffmpeg -i input.mp4 -c:v libx264 -preset slow -crf 22 -c:a copy output.mp4.

On mac - you can convert it into H.264 by right-clicking on the video file and selecting 'Encode Selected Video File' - and choosing the H.264 format.

or run the code below and store the new encoded videos in a new directory and then perform the addition of lags

In [25]:
withLagDirectory = newDirectory + 'withLag'
if not os.path.exists(withLagDirectory):
    os.mkdir(withLagDirectory)

for vid in tqdm(videos, desc = 'Videos Processed: '):

# #   Optional - depends on what files you have in the directory - check once in the previous cell's code
#     if element == '.DS_Store' or element == 'withLag':
#         continue

    video = VideoFileClip(f'{newDirectory}/{vid}')

    # Create a black screen clip with duration
    black_screen = ColorClip((video.w, video.h), color=(0, 0, 0), duration=1)

    # # Concatenate the black screen clip with the original video
    # final_clip = concatenate_videoclips([black_screen, video])

    # Append the black screen clip to the end of the video
    final_clip = CompositeVideoClip([video, black_screen.set_start(video.duration)])
    
    video_name_parts = vid.split(".")
    video_name = video_name_parts[0]
    print('\n', video_name)

#     break
    final_clip.write_videofile(f'{newDirectory}/withLag/{video_name}.mp4')

Videos Processed:   0%|                                   | 0/4 [00:00<?, ?it/s]


 good_gradual_exp_46
Moviepy - Building video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_46.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_46.mp4




Videos Processed:  25%|██████▊                    | 1/4 [00:07<00:20,  6.97s/it]

Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//withLag/good_gradual_exp_46.mp4

 good_gradual_exp_47
Moviepy - Building video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_47.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_47.mp4




Videos Processed:  50%|█████████████▌             | 2/4 [00:11<00:10,  5.39s/it]

Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//withLag/good_gradual_exp_47.mp4

 good_gradual_exp_48
Moviepy - Building video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_48.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_48.mp4




Videos Processed:  75%|████████████████████▎      | 3/4 [00:12<00:03,  3.47s/it]

Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//withLag/good_gradual_exp_48.mp4

 good_gradual_exp_49
Moviepy - Building video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_49.mp4.
Moviepy - Writing video ../stimulus_dataset/new_temp//withLag/good_gradual_exp_49.mp4




Videos Processed: 100%|███████████████████████████| 4/4 [00:14<00:00,  3.51s/it]

Moviepy - Done !
Moviepy - video ready ../stimulus_dataset/new_temp//withLag/good_gradual_exp_49.mp4


#### Calculate video durations

First move the new stimulus videos from `new_temp/withLag` to the stimulusVideos directory and then run the below code or change the directory appropriately to calculate the video durations + make sure the `Final Video Name` of the stimulusVideo is updated in teh `stimulus_dataset_information.xlsx`

In [31]:
df = pd.read_excel('../../stimulus_dataset/stimulus_dataset_information.xlsx')
stimulus_video_path = '../../stimulus_dataset/stimulusVideos/'
videos = sorted([video for video in os.listdir(stimulus_video_path) if video not in files_to_ignore])
# print(videos)
for vid in sorted(videos):
    # Load the video file
    video_path = f"../../stimulus_dataset/stimulusVideos/{vid}"
    video = VideoFileClip(video_path)

    # Get the duration of the video in seconds
    duration = video.duration
    for index, row in df.iterrows():
        if vid == row['Final Video Name']:
            df.at[index, 'Duration'] = duration

#     # Print the duration in seconds
#     print(f"Video = {vid} & duration: {duration} seconds")

#     # Optionally, you can also convert the duration to minutes and seconds
#     minutes, seconds = divmod(duration, 60)
#     print(f"Video duration: {minutes} minutes and {seconds} seconds")
df

,Stimulus Video Name,Outcome,Temporality,Anticipation,Brief Description,Human/Robot,Source,Shortlist,Comments,Final Video Name,Duration (s),When to Pause? (s),Duration
0,ch5.mp4,good,sudden,unexpected,Human avoids being drenched in water from a wave,human,SuperBAD dataset,Y,NaN,good_unexp_sudden_1.mp4,8.70,NaN,8.70
1,cr4.mp4,good,gradual,expected,Robot turns on a light switch,robot,SuperBAD dataset,Y,NaN,good_exp_gradual_2.mp4,14.12,NaN,14.12
2,fh1.mp4,bad,sudden,expected,Human falls trying to imitate pixar intro scene,human,SuperBAD dataset,M,not enough time to understand context,bad_sudden_exp_3.mp4,7.15,NaN,7.15
3,fh2.mp4,bad,sudden,expected,Human fails at performing box jumps,human,SuperBAD dataset,M,might be too obvious it goes bad,bad_sudden_exp_4.mp4,20.72,NaN,20.72
4,fh4.mp4,bad,sudden,unexpected,Falling off of a trampoline,human,SuperBAD dataset,M,we already have other trampoline videos. Maybe...,bad_sudden_unexp_5.mp4,6.77,NaN,6.77
5,fh5.mp4,bad,sudden,unexpected,Human crashes into another person on a lawnmover,human,SuperBAD dataset,Y,NaN,bad_sudden_enexp_6.mp4,7.64,NaN,7.64
6,fh7.mp4,bad,sudden,expected,Human crashes into a christman tree on a hover...,human,SuperBAD dataset,Y_p,NaN,bad_sudden_exp_7.mp4,9.84,4s,9.84
7,fh9.mp4,bad,gradual,expected,Human fails at stocking wines and causes wine ...,human,SuperBAD dataset,N,"not enough resolution, happens too suddenly",bad_gradual_exp_8.mp4,13.08,NaN,13.08
8,fr1.mp4,bad,gradual,expected,Humanoid Robot moving back and forth,robot,SuperBAD dataset,Y_p,might need to investigate where to stop video,bad_gradual_exp_9.mp4,11.43,7.5 s approx. and cut 1s from start,11.43
9,fr2.mp4,bad,sudden,unexpected,Robotic arm feeding cheeots to a mannequin,robot,SuperBAD dataset,M,I think people might not understand intention ...,bad_sudden_unexp_10.mp4,11.30,NaN,11.30


In [32]:
df.to_csv('../../stimulus_dataset/stimulus_dataset_information.csv', index = False)